In [ ]:
pip install 'spacy[transformers]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.8/795.8 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1


In [ ]:
!git clone https://github.com/roshan-research/hazm.git

# Replace Python version constraint
!sed -i 's/python= ">=3.8, <3.12"/python= ">=3.8"/g' hazm/pyproject.toml

# Replace numpy version constraint
!sed -i 's/numpy = "~1.24"/numpy = "1.26"/g' hazm/pyproject.toml

!pip install ./hazm

Cloning into 'hazm'...
remote: Enumerating objects: 8674, done.
remote: Counting objects: 100% (1695/1695), done.
remote: Compressing objects: 100% (300/300), done.
remote: Total 8674 (delta 1548), reused 1395 (delta 1395), pack-reused 6979 (from 3)
Receiving objects: 100% (8674/8674), 24.46 MiB | 27.23 MiB/s, done.
Resolving deltas: 100% (5548/5548), done.
Processing ./hazm
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293

In [ ]:
! git clone https://huggingface.co/roshan-research/spacy_pos_tagger_parsbertpostagger

Cloning into 'spacy_pos_tagger_parsbertpostagger'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21 (from 1)
Unpacking objects: 100% (21/21), 719.18 KiB | 2.34 MiB/s, done.


In [ ]:
import re

In [ ]:
import spacy
"""این ماژول شامل کلاس‌ها و توابعی برای برچسب‌گذاری توکن‌هاست."""

from nltk.tag import stanford  # noqa: I001
from hazm import SequenceTagger

import os
import subprocess

from spacy.tokens import Doc
from spacy.tokens import DocBin
from spacy.vocab import Vocab

from sklearn.metrics import classification_report,f1_score,accuracy_score,precision_score,recall_score

from tqdm import tqdm


punctuation_list = [
    '"',
    "#",
    "(",
    ")",
    "*",
    ",",
    "-",
    ".",
    "/",
    ":",
    "[",
    "]",
    "«",
    "»",
    "،",
    ";",
    "?",
    "!",
]


class POSTagger(SequenceTagger):
    """این کلاس‌ها شامل توابعی برای برچسب‌گذاری توکن‌هاست."""

    def __init__(
        self: "POSTagger", model=None, data_maker=None, universal_tag=False,
    ) -> None:
        data_maker = self.data_maker if data_maker is None else data_maker
        self.__is_universal = universal_tag
        super().__init__(model, data_maker)

    def __universal_converter(self: "POSTagger", tagged_list):
        return [(word, tag.split(",")[0]) for word, tag in tagged_list]

    def __is_punc(self: "POSTagger", word):
        return word in punctuation_list

    def data_maker(self: "POSTagger", tokens):
        """تابعی که لیستی از لیستی از کلمات توکنایز شده را گرفته و لیست دو بعدی از از دیکشنری‌هایی که تعیین‌کننده ویژگی‌ها هر کلمه هستند را برمی‌گرداند.

        Examples:
            >>> posTagger = POSTagger(model = 'pos_tagger.model')
            >>> posTagger.data_maker(tokens = [['دلم', 'اینجا', 'مانده‌است', '.']])
            [[{'word': 'دلم', 'is_first': True, 'is_last': False, 'prefix-1': 'د', 'prefix-2': 'دل', 'prefix-3': 'دلم', 'suffix-1': 'م', 'suffix-2': 'لم', 'suffix-3': 'دلم', 'prev_word': '', 'two_prev_word': '', 'next_word': 'اینجا', 'two_next_word': 'مانده\u200cاست', 'is_numeric': False, 'prev_is_numeric': '', 'next_is_numeric': False, 'is_punc': False, 'prev_is_punc': '', 'next_is_punc': False}, {'word': 'اینجا', 'is_first': False, 'is_last': False, 'prefix-1': 'ا', 'prefix-2': 'ای', 'prefix-3': 'این', 'suffix-1': 'ا', 'suffix-2': 'جا', 'suffix-3': 'نجا', 'prev_word': 'دلم', 'two_prev_word': '.', 'next_word': 'مانده\u200cاست', 'two_next_word': '.', 'is_numeric': False, 'prev_is_numeric': False, 'next_is_numeric': False, 'is_punc': False, 'prev_is_punc': False, 'next_is_punc': False}, {'word': 'مانده\u200cاست', 'is_first': False, 'is_last': False, 'prefix-1': 'م', 'prefix-2': 'ما', 'prefix-3': 'مان', 'suffix-1': 'ت', 'suffix-2': 'ست', 'suffix-3': 'است', 'prev_word': 'اینجا', 'two_prev_word': 'دلم', 'next_word': '.', 'two_next_word': '', 'is_numeric': False, 'prev_is_numeric': False, 'next_is_numeric': False, 'is_punc': False, 'prev_is_punc': False, 'next_is_punc': True}, {'word': '.', 'is_first': False, 'is_last': True, 'prefix-1': '.', 'prefix-2': '.', 'prefix-3': '.', 'suffix-1': '.', 'suffix-2': '.', 'suffix-3': '.', 'prev_word': 'مانده\u200cاست', 'two_prev_word': 'اینجا', 'next_word': '', 'two_next_word': '', 'is_numeric': False, 'prev_is_numeric': False, 'next_is_numeric': '', 'is_punc': True, 'prev_is_punc': False, 'next_is_punc': ''}]]

        Args:
            tokens (List[List[str]]): جملاتی که نیاز به تبدیل آن به برداری از ویژگی‌ها است.

        Returns:
            List(List(Dict())): لیستی از لیستی از دیکشنری‌های بیان‌کننده ویژگی‌های یک کلمه.
        """
        return [
            [self.features(token, index) for index in range(len(token))]
            for token in tokens
        ]

    def features(self: "POSTagger", sentence, index):
        """features."""
        return {
            "word": sentence[index],
            "is_first": index == 0,
            "is_last": index == len(sentence) - 1,
            # *ix
            "prefix-1": sentence[index][0],
            "prefix-2": sentence[index][:2],
            "prefix-3": sentence[index][:3],
            "suffix-1": sentence[index][-1],
            "suffix-2": sentence[index][-2:],
            "suffix-3": sentence[index][-3:],
            # word
            "prev_word": "" if index == 0 else sentence[index - 1],
            "two_prev_word": "" if index == 0 else sentence[index - 2],
            "next_word": "" if index == len(sentence) - 1 else sentence[index + 1],
            "two_next_word": (
                ""
                if index in {len(sentence) - 1, len(sentence) - 2}
                else sentence[index + 2]
            ),
            # digit
            "is_numeric": sentence[index].isdigit(),
            "prev_is_numeric": "" if index == 0 else sentence[index - 1].isdigit(),
            "next_is_numeric": (
                "" if index == len(sentence) - 1 else sentence[index + 1].isdigit()
            ),
            # punc
            "is_punc": self.__is_punc(sentence[index]),
            "prev_is_punc": "" if index == 0 else self.__is_punc(sentence[index - 1]),
            "next_is_punc": (
                ""
                if index == len(sentence) - 1
                else self.__is_punc(sentence[index + 1])
            ),
        }

    def tag(self: "POSTagger", tokens):
        """یک جمله را در قالب لیستی از توکن‌ها دریافت می‌کند و در خروجی لیستی از
        `(توکن، برچسب)`ها برمی‌گرداند.

        Examples:
            >>> posTagger = POSTagger(model = 'pos_tagger.model')
            >>> posTagger.tag(tokens = ['من', 'به', 'مدرسه', 'ایران', 'رفته_بودم', '.'])
            [('من', 'PRON'), ('به', 'ADP'), ('مدرسه', 'NOUN,EZ'), ('ایران', 'NOUN'), ('رفته_بودم', 'VERB'), ('.', 'PUNCT')]

            >>> posTagger = POSTagger(model = 'pos_tagger.model', universal_tag = True)
            >>> posTagger.tag(tokens = ['من', 'به', 'مدرسه', 'ایران', 'رفته_بودم', '.'])
            [('من', 'PRON'), ('به', 'ADP'), ('مدرسه', 'NOUN'), ('ایران', 'NOUN'), ('رفته_بودم', 'VERB'), ('.', 'PUNCT')]

        Args:
            tokens (List[str]): لیستی از توکن‌های یک جمله که باید برچسب‌گذاری شود.

        Returns:
            (List[Tuple[str,str]]): ‌لیستی از `(توکن، برچسب)`ها.

        """
        tagged_token = super().tag(tokens)
        return (
            self.__universal_converter(tagged_token)
            if self.__is_universal
            else tagged_token
        )

    def tag_sents(self: "POSTagger", sentences):
        """جملات را در قالب لیستی از توکن‌ها دریافت می‌کند
        و در خروجی، لیستی از لیستی از `(توکن، برچسب)`ها برمی‌گرداند.

        هر لیست از `(توکن، برچسب)`ها مربوط به یک جمله است.

        Examples:
            >>> posTagger = POSTagger(model = 'pos_tagger.model')
            >>> posTagger.tag_sents(sentences = [['من', 'به', 'مدرسه', 'ایران', 'رفته_بودم', '.']])
            [[('من', 'PRON'), ('به', 'ADP'), ('مدرسه', 'NOUN,EZ'), ('ایران', 'NOUN'), ('رفته_بودم', 'VERB'), ('.', 'PUNCT')]]

            >>> posTagger = POSTagger(model = 'pos_tagger.model', universal_tag = True)
            >>> posTagger.tag_sents(sentences = [['من', 'به', 'مدرسه', 'ایران', 'رفته_بودم', '.']])
            [[('من', 'PRON'), ('به', 'ADP'), ('مدرسه', 'NOUN'), ('ایران', 'NOUN'), ('رفته_بودم', 'VERB'), ('.', 'PUNCT')]]

        Args:
            sentences (List[List[str]]): لیستی از جملات که باید برچسب‌گذاری شود.

        Returns:
            (List[List[Tuple[str,str]]]): لیستی از لیستی از `(توکن، برچسب)`ها.
                    هر لیست از `(توکن،برچسب)`ها مربوط به یک جمله است.

        """
        tagged_sents = super().tag_sents(sentences)
        return (
            [self.__universal_converter(tagged_sent) for tagged_sent in tagged_sents]
            if self.__is_universal
            else tagged_sents
        )


class StanfordPOSTagger(stanford.StanfordPOSTagger):
    """StanfordPOSTagger."""

    def __init__(
        self: "StanfordPOSTagger",
        model_filename: "str",
        path_to_jar: str,
        *args, # noqa: ANN002
        **kwargs, # noqa: ANN003
    ) -> None:
        self._SEPARATOR = "/"
        super(stanford.StanfordPOSTagger, self).__init__(
            model_filename=model_filename,
            path_to_jar=path_to_jar,
            *args,  # noqa: B026
            **kwargs,
        )

    def tag(self: "StanfordPOSTagger", tokens):
        """tag.

        Examples:
            >>> tagger = StanfordPOSTagger(model_filename='persian.tagger', path_to_jar='stanford_postagger.jar')
            >>> tagger.tag(['من', 'به', 'مدرسه', 'رفته_بودم', '.'])
            [('من', 'PRO'), ('به', 'P'), ('مدرسه', 'N'), ('رفته_بودم', 'V'), ('.', 'PUNC')]

        """
        return self.tag_sents([tokens])[0]

    def tag_sents(self: "StanfordPOSTagger", sentences):
        """tag_sents."""
        refined = ([w.replace(" ", "_") for w in s] for s in sentences)
        return super(stanford.StanfordPOSTagger, self).tag_sents(refined)


class SpacyPOSTagger(POSTagger):
    def __init__(
        self: "SpacyPOSTagger",
        model_path=None,
        using_gpu=False,
        gpu_id=0
    ):
        """
        Initialize the SpacyPOSTagger with a model and data paths.

        Args:
        - model_path: Path to a pre-trained spaCy model.
        - test_dataset: Test dataset for evaluation. It has a similar structure to the training dataset.
        - test_directory: Directory to save the test data in spaCy format.
        - using_gpu: Set to True if you want use gpu (if you dont have one and set this to True the function use cpu automatically)
        This constructor calls the constructor of the parent class POSTagger.
        """
        super().__init__(universal_tag=True)
        self.model_path = model_path #### Usually an output directory for spacy model contain two other directory name model-last , model-best,You should give model_path like this : output/model-last
        self.using_gpu = using_gpu
        self.gpu_id = gpu_id
        self.tagger = None
        self._setup()

    def _setup(self: "SpacyPOSTagger"):
        """
        Set up the configuration for the spaCy model, including GPU settings and data preparation.

        This function initializes and configures the spaCy model, checks for GPU availability, and prepares the training and testing datasets in spaCy format.

        If using GPU, GPU settings are configured to enhance processing speed. Then, the pre-trained spaCy model is loaded based on the provided model path.

        Training and testing datasets are prepared and saved in the respective directories for use during model training and evaluation.
        """  # noqa: D212
        if self.using_gpu:
            self._setup_gpu()
        else:
            print("------------- You Prefer to use CPU --------------")


    def _setup_model(self: "SpacyPOSTagger",sents):
        """
        Initialize and configure the spaCy model for tagging without GPU settings.

        This method loads and configures the spaCy model based on the provided model path. It also sets up a custom tokenizer for text processing and constructs a dictionary for reference.

        Args:
            - model_path: Path to the pre-trained spaCy model.

        This method is typically called during setup to prepare the model for tagging tasks.
        """
        self.peykare_dict = {}  # Initialize a dictionary for reference
        self.tagger = spacy.load(self.model_path)  # Load the spaCy model
        self._set_peykare_dictionary(sents)  # Construct a reference dictionary
        self.tagger.tokenizer = self._custom_tokenizer  # Set a custom tokenizer for the model.

    def _setup_gpu(self: "SpacyPOSTagger"):
        """
        Check GPU availability and configure spaCy to use it if possible.

        This method checks whether a GPU is available and, if so, configures spaCy to utilize it for improved processing speed. It sets the 'gpu_availability' attribute to 'True' or 'False' accordingly.

        This check is performed during setup to make use of available GPU resources for enhanced performance.
        """
        print("------------------ GPU Setup Process Started ---------------------")
        gpu_available = spacy.prefer_gpu(gpu_id=self.gpu_id)  # Check if a GPU is available
        if gpu_available:
            print("------------ GPU is available and ready for use -------------")
            spacy.require_gpu(gpu_id=self.gpu_id)  # Configure spaCy to use the GPU
            self.gpu_availability = True
        else:
            print("------------ GPU is not available; spaCy will use CPU -------------")
            self.gpu_availability = False


    def _setup_dataset(self: "SpacyPOSTagger", dataset,saved_directory,data_type='train'):
        """
        Setup the training dataset in spaCy's binary format.

        This function prepares the training dataset and saves it in spaCy's binary format.
        """
        assert data_type in ['train','test']
        db = DocBin()
        for sent in tqdm(dataset):
            words = [word[0] for word in sent]
            tags = [word[1] for word in sent]
            doc = Doc(Vocab(strings=words), words=words)
            for d, tag in zip(doc, tags):
                d.tag_ = tag
            db.add(doc)

        self._handle_data_path(saved_directory)
        db.to_disk(f'{saved_directory}/{data_type}.spacy')

    def _handle_data_path(self,path='POSTaggerDataset'):
        """
        Create the directory if it doesn't exist.

        This method checks if the specified directory exists, and if not, it creates the directory to store the data.

        Args:
            - path: The path to the directory (default is 'POSTaggerDataset').

        This method is called to ensure the directory is available for saving processed data.
        """
        if not os.path.exists(path):
            os.makedirs(path)


    def _custom_tokenizer(self: "SpacyPOSTagger", text):
        """
        Implement a custom tokenization method for spaCy.

        This method defines a custom tokenization method for spaCy. It is used to tokenize input text based on a custom dictionary, or it raises an error if tokenization is not available.

        Args:
            - text: The input text to be tokenized.

        This custom tokenization method is used by the spaCy model during processing.

        """

        if text in self.peykare_dict:
            return Doc(self.tagger.vocab, self.peykare_dict[text])
        else:
            raise ValueError('No tokenization available for input.')

    def _set_peykare_dictionary(self: "SpacyPOSTagger", sents):
        """
        Create a dictionary for custom tokenization.

        This method constructs a dictionary to store custom tokenization mappings based on input sentences. It is used for custom tokenization in spaCy.

        Args:
            - sents: Input sentences to build the custom tokenization dictionary.

        This method is called during setup to establish a dictionary for tokenization.
        """
        self.peykare_dict = {' '.join([w for w in item]): [w for w in item] for item in sents}


    def _add_to_dict(self: "SpacyPOSTagger", sents):
        """
            Add the sentences to dictianory if it doesnt exist already
        """
        for sent in sents:
            key = ' '.join(sent)
            if key not in self.peykare_dict:
                self.peykare_dict[key] = sent


    def tag(self: "SpacyPOSTagger", tokens,universal_tag=True):
        """یک جمله را در قالب لیستی از توکن‌ها دریافت می‌کند و در خروجی لیستی از
        `(توکن، برچسب)`ها برمی‌گرداند.

        Examples:
            >>> posTagger = POSTagger(model = 'pos_tagger.model')
            >>> posTagger.tag(tokens = ['من', 'به', 'مدرسه', 'ایران', 'رفته_بودم', '.'])
            [('من', 'PRON'), ('به', 'ADP'), ('مدرسه', 'NOUN,EZ'), ('ایران', 'NOUN'), ('رفته_بودم', 'VERB'), ('.', 'PUNCT')]

            >>> posTagger = POSTagger(model = 'pos_tagger.model', universal_tag = True)
            >>> posTagger.tag(tokens = ['من', 'به', 'مدرسه', 'ایران', 'رفته_بودم', '.'])
            [('من', 'PRON'), ('به', 'ADP'), ('مدرسه', 'NOUN'), ('ایران', 'NOUN'), ('رفته_بودم', 'VERB'), ('.', 'PUNCT')]

        Args:
            tokens (List[str]): لیستی از توکن‌های یک جمله که باید برچسب‌گذاری شود.

        Returns:
            (List[Tuple[str,str]]): ‌لیستی از `(توکن، برچسب)`ها.

        """
        if self.tagger == None:
            self._setup_model([tokens])
        self._add_to_dict([tokens])

        text = ' '.join(tokens)
        doc = self.tagger(text)
        if not universal_tag:
            tags = [tok.tag_ for tok in doc]
        else:
            tags = [tok.tag_.replace(',EZ','') for tok in doc]

        return list(zip(tokens,tags))
              # noqa: W293

    def tag_sents(self:"SpacyPOSTagger",sents,universal_tag=True,batch_size=128):
        """
            Args:
                sents : List[List[Tokens]]
                batch_size : number of batches give to model for processing sentences each time
        """
        """
            Returns : List[List[Tuple(str,str)]]
        """
        if self.tagger == None:
            self._setup_model(sents)

        self._add_to_dict(sents)

        docs = list(self.tagger.pipe((' '.join([w for w in sent]) for sent in sents), batch_size=batch_size))
        if not universal_tag:
            tags = [[w.tag_ for w in doc] for doc in docs]
        else:
            tags = [[w.tag_.replace(',EZ','') for w in doc] for doc in docs]

        combined_out = [list(zip(tok,tag)) for tok,tag in zip(sents,tags)]
        return combined_out

    def train(
        self: "SpacyPOSTagger",
        train_dataset,
        test_dataset,
        data_directory,
        base_config_file,
        train_config_path,
        output_dir,
        use_direct_config=False
    ):
        """
        Train the spaCy model using a subprocess and a configuration file.

        This method executes the training process for the spaCy model by invoking spaCy's training module using subprocess. It takes input configuration files, training and testing data, and GPU settings (if available).

        Args:
            - train_dataset: Training dataset for the tagger. It is a list of sentences, where each sentence is a list of token-tag pairs.
            - test_dataset: Testing dataset for the tagger. It is a list of sentences, where each sentence is a list of token-tag pairs.
            - data_directory: Directory to save the training and testing data in spaCy format.
            - base_config_file: Path to the base configuration file for spaCy.
            - train_config_path: if use_direct_config set to True this is the path of config file for training that you will use
              if use_direct_config set to False this is the path that you want train config file will create with base_config
            - output_dir: Directory for storing the trained model and training logs.

        Upon successful training, this method updates the model path to the trained model.

        This method is typically called to initiate the training process of the spaCy model.
        """

        self.spacy_train_directory = data_directory
        self.train_dataset = train_dataset ### List[List[Tuple]]
        self.test_dataset = test_dataset
        if self.train_dataset:
            # Set up the training dataset configuration
            self._setup_dataset(dataset=self.train_dataset, saved_directory=self.spacy_train_directory, data_type='train')

        if self.test_dataset:
            self._setup_dataset(test_dataset,saved_directory=data_directory,data_type='test')

        train_data = f'{data_directory}/train.spacy'
        test_data = f'{data_directory}/test.spacy'

        if use_direct_config == False:
            self._setup_train_config(base_config_file, train_config_path=train_config_path)
        else:
            self.train_config_file = train_config_path

        command = f"python -m spacy train {self.train_config_file} --output ./{output_dir} --paths.train ./{train_data} --paths.dev ./{test_data}"
        if self.gpu_availability:
            command += f" --gpu-id {self.gpu_id}"

        subprocess.run(command, shell=True)
        self.model_path = f"{output_dir}/model-last"
        self._setup_model([[w for w,_ in sent] for sent in test_dataset])

    def _setup_train_config(self: "SpacyPOSTagger", base_config, train_config_path):
        """
        Create and configure the training configuration file for spaCy.

        This method sets up the training configuration file by copying a base configuration file and customizing it according to the specified parameters.

        Args:
            - base_config: Path to the base configuration file.
            - train_config_file_name: Name of the training configuration file for saving it.

        This method is called to generate the training configuration file used in the training process.
        """
        self.train_config_file = train_config_path
        print("----------------- Setting up the training configuration file ----------------------")
        command = f"python -m spacy init fill-config {base_config} {train_config_path}"  # Generate the training configuration file
        subprocess.run(command, shell=True)
        print("----------------- Training configuration file created successfully ----------------------")
        print(f"----------------- Training Config file address is {train_config_path} --------------------")

    def evaluate(self: "SpacyPOSTagger", test_sents,batch_size):
        """
        Evaluate the spaCy model on input sentences using different tag options.

        This method evaluates the spaCy model on input sentences with and without 'EZ' tags and reports classification metrics.

        Args:
            - sents: List of sentences for evaluation.
            - batch_size : number of batches that model should process each time
        This method calls the internal evaluation method for both tag options.

        This method is typically used for model evaluation and reporting metrics.
        """
        self._setup_model([[w for w,_ in sent] for sent in test_sents])
        if self.tagger:
            golds, predictions = self._get_labels_and_predictions(test_sents,batch_size)
            print("-----------------------------------------")
            self._evaluate_tags(test_sents, golds, predictions, use_ez_tags=True,batch_size=batch_size)
            print("-----------------------------------------")
            self._evaluate_tags(test_sents, golds, predictions, use_ez_tags=False,batch_size=batch_size)
        else:
            raise ValueError("Model does not exist.Please train a new one with train method of this instance or give a model_path , setup the model with self._setup_model() and then call evaluate")

    def _evaluate_tags(self, sents, golds=None, predictions=None, use_ez_tags=True,batch_size=128):
        """
        Evaluate model predictions and report classification metrics.

        This method evaluates model predictions and reports classification metrics for the specified tag option.

        Args:
            - sents: List of sentences for evaluation.
            - golds: List of gold labels (optional).
            - predictions: List of model predictions (optional).
            - use_ez_tags: A flag indicating whether to consider 'EZ' tags.
            - batch_size : number of batches model should process

        If `golds` and `predictions` are not provided, they are automatically extracted from the input sentences.

        This method calculates and displays precision, recall, and F1-score for the specified tag option.

        This method is called by the `evaluate` method to perform model evaluation.
        """
        if golds is None or predictions is None:
            golds, predictions = self._get_labels_and_predictions(sents,batch_size)

        predictions_cleaned = []
        golds_cleaned = []
        if use_ez_tags:
            get_tag_func = self._get_ez_tags
        else:
            get_tag_func = self._remove_ez_tags

        for preds, golds in zip(predictions, golds):
            for pred in preds:
                pred_cleaned = get_tag_func(pred)
                predictions_cleaned.append(pred_cleaned)
            for gold in golds:
                gold_cleaned = get_tag_func(gold)
                golds_cleaned.append(gold_cleaned)

        print(classification_report(golds_cleaned, predictions_cleaned))
        print('Precision: %.5f' % precision_score(golds_cleaned, predictions_cleaned, average='weighted'))
        print('Recall: %.5f' % recall_score(golds_cleaned, predictions_cleaned, average='weighted'))
        print('F1-Score: %.5f' % f1_score(golds_cleaned, predictions_cleaned, average='macro'))

    def _get_ez_tags(self, label):
        """
        Extract 'EZ' tags from labels.

        This method extracts 'EZ' tags from labels if they are present and returns them.

        Args:
            - label: The label containing 'EZ' tags.

        Returns:
            The 'EZ' tags or '-' if 'EZ' tags are not present.
        """
        if 'EZ' in label:
            label = 'EZ'
        else:
            label = '-'

        return label

    def _remove_ez_tags(self, label):
        """
        Remove 'EZ' tags from labels.

        This method removes 'EZ' tags from labels if they are present and returns the cleaned label.

        Args:
            - label: The label containing 'EZ' tags.

        Returns:
            The label with 'EZ' tags removed.
        """
        return label.replace(',EZ', '') if 'EZ' in label else label

    def _evaluate_ez_tags(self, sents):
        """
        Evaluate model predictions with 'EZ' tags included.

        This method evaluates model predictions with 'EZ' tags included.
        """
        self._evaluate_tags(sents, use_ez_tags=True)

    def _evaluate_normal_tags(self, sents):
        """
        Evaluate model predictions without 'EZ' tags.

        This method evaluates model predictions without 'EZ' tags.
        """
        self._evaluate_tags(sents, use_ez_tags=False)

    def _get_labels_and_predictions(self: "SpacyPOSTagger", sents,batch_size):
        """
        Extract gold labels and model predictions for evaluation.

        This method extracts gold labels and model predictions from input sentences.

        Args:
            - sents: List of sentences for evaluation.

        Returns:
            Lists of gold labels and model predictions.

        This method is typically used for gathering data to perform model evaluation.
        """
        gold_labels = [[tag for _, tag in sent] for sent in sents]
        tokens = [[w for w,_ in sent] for sent in sents]
        prediction_labels = self.tag_sents(tokens,batch_size)
        return gold_labels, prediction_labels


In [ ]:
import os
import psutil
import torch
import spacy

def analyze_spacy_model(model_path):
    """
    Reports detailed size and memory information about a spaCy model,
    including:
      - number of trainable parameters
      - memory footprint of weights
      - model size on disk
      - RAM used when loaded
    Works for both Thinc and HuggingFace Transformer backends.
    """

    print(f"📦 Loading model from: {model_path}")
    process = psutil.Process(os.getpid())
    before_mem = process.memory_info().rss / (1024 * 1024)

    nlp = spacy.load(model_path)

    after_mem = process.memory_info().rss / (1024 * 1024)
    load_mem = after_mem - before_mem

    print(f"✅ Model loaded. Memory used during load: {load_mem:.2f} MB\n")

    total_params = 0
    total_weight_bytes = 0

    for name, component in nlp.pipeline:
        print(f"🔹 Component: {name} -> {component.__class__.__name__}")

        torch_model = None
        if hasattr(component, "model"):
            model_attr = component.model

            # Try to find a torch model
            if hasattr(model_attr, "transformer"):
                inner = model_attr.transformer
                torch_model = getattr(inner, "model", inner)
            elif hasattr(model_attr, "model"):
                torch_model = model_attr.model
            elif isinstance(model_attr, torch.nn.Module):
                torch_model = model_attr

            # Count parameters and weight memory if torch model found
            if isinstance(torch_model, torch.nn.Module):
                params = sum(p.numel() for p in torch_model.parameters())
                weight_bytes = sum(p.nelement() * p.element_size() for p in torch_model.parameters())
                print(f"   ➜ Transformer parameters: {params:,}")
                print(f"   ➜ Weight memory: {weight_bytes / (1024 * 1024):.2f} MB\n")
                total_params += params
                total_weight_bytes += weight_bytes
                continue

            # Otherwise, try Thinc fallback
            try:
                weights = list(model_attr.walk_params())
                params = sum(w.size for _, w in weights)
                weight_bytes = params * 4  # assume float32
                print(f"   ➜ Non-transformer parameters: {params:,}")
                print(f"   ➜ Weight memory: {weight_bytes / (1024 * 1024):.2f} MB\n")
                total_params += params
                total_weight_bytes += weight_bytes
            except Exception:
                print("   ➜ Could not extract parameters.\n")
        else:
            print("   ➜ No model found in this component.\n")

    # Compute model directory size
    total_disk_bytes = sum(
        os.path.getsize(os.path.join(dp, f))
        for dp, _, filenames in os.walk(model_path)
        for f in filenames
    )
    total_disk_mb = total_disk_bytes / (1024 * 1024)

    print("📊 SUMMARY")
    print("-" * 60)
    print(f"🧠 Total trainable parameters : {total_params:,}")
    print(f"💾 Total weight memory (float32): {total_weight_bytes / (1024 * 1024):.2f} MB")
    print(f"📂 Model size on disk          : {total_disk_mb:.2f} MB")
    print(f"⚙️  Memory used after load      : {load_mem:.2f} MB")
    print("-" * 60)

    return {
        "parameters": total_params,
        "weight_memory_MB": total_weight_bytes / (1024 * 1024),
        "disk_size_MB": total_disk_mb,
        "load_memory_MB": load_mem
    }


# ---- Run the analyzer ----
if __name__ == "__main__":
    model_path = "/content/spacy_pos_tagger_parsbertpostagger"
    report = analyze_spacy_model(model_path)


📦 Loading model from: /content/spacy_pos_tagger_parsbertpostagger


/usr/local/lib/python3.12/dist-packages/spacy/util.py:922: UserWarning: [W095] Model 'fa_pipeline' (0.0.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.8.7). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/spacy_transformers/layers/hf_shim.py:137: UserWarning: Error loading saved torch state_dict with strict=True, likely due to differences between 'transformers' versions. Attempting to load with strict=False as a fallback...

If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current 'transformers' and 'spacy-transformers' versions. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


✅ Model loaded. Memory used during load: 311.93 MB

🔹 Component: transformer -> Transformer
   ➜ Transformer parameters: 162,841,344
   ➜ Weight memory: 621.19 MB

🔹 Component: tagger -> Tagger
   ➜ Could not extract parameters.

📊 SUMMARY
------------------------------------------------------------
🧠 Total trainable parameters : 162,841,344
💾 Total weight memory (float32): 621.19 MB
📂 Model size on disk          : 1258.49 MB
⚙️  Memory used after load      : 311.93 MB
------------------------------------------------------------


In [ ]:
# Your instance (assuming the model path is valid and the model contains a 'tagger')
spacy_posTagger = SpacyPOSTagger(model_path = '/content/spacy_pos_tagger_parsbertpostagger')

------------- You Prefer to use CPU --------------


In [ ]:
from hazm import WordTokenizer, Normalizer

tokenizer = WordTokenizer()
normalizer = Normalizer()

In [ ]:
def get_tags(grapheme, compound=False, tokens=None):
  grapheme = re.sub('ۀ', 'ه‌ی', grapheme)
  if not tokens:
    tokens = tokenizer.tokenize(normalizer.normalize(grapheme))
  tags = spacy_posTagger.tag(tokens=tokens, universal_tag=False)
  return tags

# Performance Test

In [ ]:
import pandas as pd
import ast


def compute_ezafe_confusion(csv_path):
    df = pd.read_csv(csv_path)

    # Initialize counts
    tp = tn = fp = fn = 0

    for i, row in df.iterrows():
        sentence = row['grapheme']
        gold_tags = ast.literal_eval(row['ezafe_tags'])
        predicted_tags = get_tags(sentence)

        # Extract gold and predicted Ezafe Booleans (in token order)
        gold_dict = {word: ez for word, ez in gold_tags}
        pred_dict = {word: ('EZ' in tag) for word, tag in predicted_tags}

        # Compare token by token (based on the gold tokens order)
        for word, gold_ez in gold_dict.items():
            pred_ez = pred_dict.get(word, False)  # default False if missing
            if gold_ez and pred_ez:
                tp += 1
            elif not gold_ez and not pred_ez:
                tn += 1
            elif not gold_ez and pred_ez:
                fp += 1
            elif gold_ez and not pred_ez:
                fn += 1

    print(f"True Positives (TP): {tp}")
    print(f"True Negatives (TN): {tn}")
    print(f"False Positives (FP): {fp}")
    print(f"False Negatives (FN): {fn}")
    print(f"Total tokens: {tp + tn + fp + fn}")
    if tp + fp > 0:
        print(f"Precision: {tp / (tp + fp):.4f}")
    if tp + fn > 0:
        print(f"Recall: {tp / (tp + fn):.4f}")
    if tp + fp + fn > 0:
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        if precision + recall > 0:
            print(f"F1 Score: {2 * precision * recall / (precision + recall):.4f}")

# Example usage
compute_ezafe_confusion("augSentenceBench.csv")


True Positives (TP): 545
True Negatives (TN): 2616
False Positives (FP): 21
False Negatives (FN): 5
Total tokens: 3187
Precision: 0.9629
Recall: 0.9909
F1 Score: 0.9767


# Speed Test

# Get Evaluation Data

In [ ]:
import pandas as pd

In [ ]:
!wget https://huggingface.co/datasets/MahtaFetrat/SentenceBench/raw/main/SentenceBench.csv

--2025-11-07 14:45:49--  https://huggingface.co/datasets/MahtaFetrat/SentenceBench/raw/main/SentenceBench.csv
Resolving huggingface.co (huggingface.co)... 3.165.160.11, 3.165.160.59, 3.165.160.61, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56026 (55K) [text/plain]
Saving to: ‘SentenceBench.csv.1’

SentenceBench.csv.1 100%[===================>]  54.71K  --.-KB/s    in 0.008s  

2025-11-07 14:45:49 (7.03 MB/s) - ‘SentenceBench.csv.1’ saved [56026/56026]



In [ ]:
sentence_bench = pd.read_csv('SentenceBench.csv')

In [ ]:
sentence_bench.head(3)

,dataset,grapheme,phoneme,homograph word,pronunciation
0,homograph,من قدر تو را می‌دانم,man qadr-e to rA mi-dAnam,قدر,qadr
1,homograph,از قضای الهی به قدر الهی پناه می‌برم,?az qazAy ?elAhi be qadar-e ?elAhi panAh mi-baram,قدر,qadar
2,homograph,به دست و صورتم کرم زدم,be dast-o suratam kerem zadam,کرم,kerem


### Get ManaTTS

In [ ]:
filtered_rows = sentence_bench[sentence_bench['dataset'] == 'mana-tts'][['grapheme', 'phoneme']]

# Convert to a list of tuples
mana_evaluation_data = list(filtered_rows.itertuples(index=False, name=None))

mana_evaluation_data[:3]

[('در این نوشته بنا داریم با یک ابزار ساده و مکانیکی افزایش بینایی برای افراد کم\u200cبینا ',
  'dar ?in neveSte banA dArim bA yek ?abzAr-e sAde va mekAniki-ye ?afzAyeS-e binAyi barAye ?afrAd-e kam\u200cbinA '),
 ('به نام بی\u200cوپتیک یا عدسی دورنما آشنا شویم. ',
  'be nAm-e biyoptik yA ?adasi-ye durnamA ?ASnA Savim'),
 ('دراین\u200cصورت، انجام خودارزیابی و ارائه بازخورد بر عهده خودتان است. ',
  'dar ?in surat ?anjAm-e xod?arzyAbi va ?erA?e-ye bAzxord bar ?ohde-ye xodetAn ?ast ')]

### Get CommonVoice

In [ ]:
filtered_rows = sentence_bench[sentence_bench['dataset'] == 'commonvoice'][['grapheme', 'phoneme']]

# Convert to a list of tuples
commonvoice_evaluation_data = list(filtered_rows.itertuples(index=False, name=None))

commonvoice_evaluation_data[:3]

[('در اکثر شهرها، مرکزی برای خرید دوچرخه وجود دارد.',
  'dar ?aksar-e Sahr-hA, markazi barAye xarid-e  doCarxe vojud dArad.'),
 ('پس از مدرسه کودکان به سوی خانه جست و خیز کردند.',
  'pas ?az madrese kudakAn be suye xAne jast-o-xiz kardand.'),
 ('شما نگران زن و بچه این نباش.', 'SomA negarAn-e zan-o-baCCe-ye ?in nabAS.')]

### Get Homograph

In [ ]:
filtered_rows = sentence_bench[sentence_bench['dataset'] == 'homograph'][['grapheme', 'phoneme', 'homograph word',	'pronunciation']]

# Convert to a list of tuples
homograph_evaluation_data = list(filtered_rows.itertuples(index=False, name=None))

homograph_evaluation_data[:3]

[('من قدر تو را می\u200cدانم', 'man qadr-e to rA mi-dAnam', 'قدر', 'qadr'),
 ('از قضای الهی به قدر الهی پناه می\u200cبرم',
  '?az qazAy ?elAhi be qadar-e ?elAhi panAh mi-baram',
  'قدر',
  'qadar'),
 ('به دست و صورتم کرم زدم', 'be dast-o suratam kerem zadam', 'کرم', 'kerem')]

# Full bench

In [ ]:
benchmark = []

for g, p in mana_evaluation_data:
  benchmark.append((g, p, '', ''))

for g, p in commonvoice_evaluation_data:
  benchmark.append((g, p, '', ''))

for g, p, w, r in homograph_evaluation_data:
  benchmark.append((g, p, w, r))

# Inference

In [ ]:
import time

start_time = time.time()

outputs = [get_tags(item[0]) for item in benchmark]

total_time = time.time() - start_time

In [ ]:
avg_time = total_time / len(outputs) if len(outputs) > 0 else 0

In [ ]:
print(f"AVG TIME:\t\t{avg_time:.4f} (s)+")

AVG TIME:		0.1105 (s)+


# Runs

## First
```
True Positives (TP): 545
True Negatives (TN): 2616
False Positives (FP): 21
False Negatives (FN): 5
Total tokens: 3187
Precision: 0.9629
Recall: 0.9909
F1 Score: 0.9767

AVG TIME:		0.1046 (s)+
```


## Second
```
True Positives (TP): 545
True Negatives (TN): 2616
False Positives (FP): 21
False Negatives (FN): 5
Total tokens: 3187
Precision: 0.9629
Recall: 0.9909
F1 Score: 0.9767

AVG TIME:		0.1116 (s)+
```


## Third
```
True Positives (TP): 545
True Negatives (TN): 2616
False Positives (FP): 21
False Negatives (FN): 5
Total tokens: 3187
Precision: 0.9629
Recall: 0.9909
F1 Score: 0.9767

AVG TIME:		0.1105 (s)+
```



In [ ]:
import numpy as np
import pandas as pd

# Store your three runs here
runs = [
    {
        "TP": 545,
        "TN": 2616,
        "FP": 21,
        "FN": 5,
        "Precision": 0.9629,
        "Recall": 0.9909,
        "F1": 0.9767,
        "AVG_TIME": 0.1046,
    },
    {
        "TP": 545,
        "TN": 2616,
        "FP": 21,
        "FN": 5,
        "Precision": 0.9629,
        "Recall": 0.9909,
        "F1": 0.9767,
        "AVG_TIME": 0.1116,
    },
    {
        "TP": 545,
        "TN": 2616,
        "FP": 21,
        "FN": 5,
        "Precision": 0.9629,
        "Recall": 0.9909,
        "F1": 0.9767,
        "AVG_TIME": 0.1105,
    },
]

# Convert to DataFrame
df = pd.DataFrame(runs)

# === Compute Accuracy for each run ===
df["Accuracy"] = (df["TP"] + df["TN"]) / (df["TP"] + df["TN"] + df["FP"] + df["FN"])

# === Compute mean and std for all metrics ===
mean_vals = df.mean()
std_vals = df.std(ddof=1)

# === Pretty-print the report ===
print("=== Ezafe Detection Evaluation Report ===\n")
print("Averaged over 3 runs\n")

metrics_order = ["TP", "TN", "FP", "FN", "Accuracy", "Precision", "Recall", "F1", "AVG_TIME"]

for metric in metrics_order:
    mean_ = mean_vals[metric]
    std_ = std_vals[metric]

    if metric in ["Precision", "Recall", "F1", "Accuracy"]:
        print(f"{metric:10s}: {mean_:.4f} ± {std_:.4f}")
    elif metric == "AVG_TIME":
        print(f"{metric:10s}: {mean_:.4f} ± {std_:.4f}  (seconds)")
    else:
        print(f"{metric:10s}: {mean_:.0f} ± {std_:.0f}")

=== Ezafe Detection Evaluation Report ===

Averaged over 3 runs

TP        : 545 ± 0
TN        : 2616 ± 0
FP        : 21 ± 0
FN        : 5 ± 0
Accuracy  : 0.9918 ± 0.0000
Precision : 0.9629 ± 0.0000
Recall    : 0.9909 ± 0.0000
F1        : 0.9767 ± 0.0000
AVG_TIME  : 0.1089 ± 0.0038  (seconds)
